In [1]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import pandas as pd

In [ ]:
df = pd.read_csv("../2.text_preprocessing/cna_FINAL.csv")
df

In [2]:
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

In [3]:
#load model
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-cnn_dailymail")

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [4]:
text = 'BERLIN (AP) — The German economy shrank by 2.2% in the first quarter compared with the previous three-month period as shutdowns in the country and beyond started to bite, official data showed Friday. That means Europe’s biggest economy went into recession following a small dip at the end of last year. The figures from the Federal Statistical Office offered a first glimpse of the damage caused by the coronavirus crisis to Europe’s biggest economy, which the government is trying to limit with a raft of rescue programs. The decline in the January-March period was the biggest since 2009. It followed a 0.3% gain in last year’s third quarter and a 0.1% contraction in the fourth quarter — the latter figure revised down from the initial report in February of zero growth. That revision put Germany into a technical recession, defined as two consecutive quarters of negative growth, in the first quarter. March was the month in which the coronavirus pandemic hit Europe, with first Italy and then other countries imposing sweeping restrictions on public life and businesses. Germany itself started shutting down in mid-March. It never ordered factories closed, but companies did largely stop production in some areas — such as the automaking sector — and supply chains were disrupted. Recent data showed a 15.6% month-on-month decrease in factory orders in March, and a 9.2% drop in industrial production. The country started loosening restrictions on April 20 and the process has gathered pace recently. Shops have now reopened, restaurants are gradually opening up and auto production has restarted.'

In [5]:
#create tokens - number representation of our text
tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")

In [6]:
summary = model.generate(**tokens)

/Users/christine/anaconda3/lib/python3.10/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
summary

tensor([[    0,   139,  5088,   115,   109,  1268,   121, 11365,   908,   140,
           109,  1715,   381,  3418,   110,   107,   106,   362,  1734,   114,
         63404,  1830,   115,   289,   232,   123,   116,   776,  2349,   111,
           114, 55898, 26357,   115,   109,  2868,  2349,   110,   107,     1]])

In [9]:
tokenizer.decode(summary[0])

'The decline in the January-March period was the biggest since 2009.<n>It followed a 0.3% gain in last year’s third quarter and a 0.1% contraction in the fourth quarter.'

In [15]:
# openai
import requests
api_key = "929e58a0-77e9-41b4-a4c4-b40a77b8bcf1"
url = "https://api.oneai.com/api/v0/pipeline"
headers = {"api-key": api_key, "content-type": "application/json"}

payload = {"text": text, "input_type":"article","steps":[{"skill":"summarize","params":{"find_origins":True, "max_length":100}}]}
r = requests.post(url, json=payload, headers=headers)
data = r.json()
print(data['output'][0]['text'])

Germany's economy shrank by 2.2% in the first quarter compared with the previous three-month period. That means Europe's biggest economy went into recession following a small dip at the end of last year. March was the month in which the coronavirus pandemic hit Europe, with first Italy and then other countries imposing sweeping restrictions. Germany started shutting down in mid-March. It never ordered factories closed, but companies did largely stop production in some areas.


In [17]:
!pip install rouge-metric
from rouge_metric import PyRouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 2.4 MB/s eta 0:00:00a 0:00:01


In [18]:
model_out = ["German economy shrank by 2.2% in the first quarter compared with the previous three-month period. That means Europe’s biggest economy went into recession following a small dip at the end of last year. The decline in the January-March period was the biggest since 2009."]


reference = ['Germany in recession as economy shrinks 2.2% in first quarter']


In [19]:
# Evaluate document-wise ROUGE scores
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
scores = rouge.evaluate(model_out, reference)

In [20]:
print(scores)

{'rouge-1': {'r': 0.057692307692307696, 'p': 0.0011177347242921013, 'f': 0.0021929824561403508}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.057692307692307696, 'p': 0.0011177347242921013, 'f': 0.0021929824561403508}, 'rouge-w-1.2': {'r': 0.09281074509222681, 'p': 0.0018466028716864877, 'f': 0.0036211576195015823}, 'rouge-s4': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-su4': {'r': 0.0, 'p': 0.0, 'f': 0.0}}


In [21]:
summary = [
    'The decline in the January-March period was the biggest since 2009.<n>It followed a 0.3% gain in last year’s third quarter and a 0.1% contraction in the fourth quarter.'
]
reference = [[
    'Germany in recession as economy shrinks 2.2% in first quarter'
]]

# Evaluate document-wise ROUGE scores
rouge = PyRouge(rouge_n=(1, 2, 3, 4), rouge_l=True)
scores = rouge.evaluate(summary, gold_truth)
print(scores)

{'rouge-1': {'r': 0.3, 'p': 0.10714285714285714, 'f': 0.15789473684210525}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.3, 'p': 0.10714285714285714, 'f': 0.15789473684210525}}


In [ ]:
"""
1. extractive -> bert
2. 2 abstractive-> pegasus          (if pegasus>extractive)then(one ai (max_length=100))  

continue on monday$
3. gold truth (from chat gpt)
4. rogue for extractive, bart, one ai vs gold truth
5. determine better model


reference means gold truth
"""

In [ ]:
"""
sya 
1. clean text
2. combine cnn and cna


"""